# CNN Autoencoder on fashion mnist

Only major differences to `cnn_autoencover_fastai2022.ipynb` is the cell loading the data, loading Fashion MNIST pictures of clothing instead of MNIST digits. The same model as for MNIST seems also fit Fashion MNIST relatively well.

## References

* fastai 2022 / 2023 course part II:
    * [notebook 8](https://github.com/fastai/course22p2/blob/master/nbs/08_autoencoder.ipynb)
    * [lesson 15](https://course.fast.ai/Lessons/lesson15.html)

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random
import re

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import tqdm
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from torch.optim import SGD
from torch.utils.data import DataLoader, Dataset

import random_neural_net_models.autoencoder_fastai2022 as ae
import random_neural_net_models.convolution_lecun1990 as conv_lecun1990
import random_neural_net_models.telemetry as telemetry

sns.set_theme()

In [ ]:
DO_OVERFITTING_ONLY = True

In [ ]:
mnist = fetch_openml("Fashion-MNIST", version=1, cache=True, parser="auto")

Setting seeds

In [ ]:
torch.manual_seed(42)

random.seed(42)

np.random.seed(42)

Getting device

In [ ]:
def get_device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"


device = get_device()
device

In [ ]:
X = mnist["data"]
y = mnist["target"]
X.shape, y.shape

Selecting a few images to overfit on

In [ ]:
n = 1
X0, y0 = X.iloc[:n], y.iloc[:n]
X0.shape

## Defining dataset and dataloader

In [ ]:
ds = conv_lecun1990.DigitsDataset(X0, y0)

In [ ]:
item = ds[0]
plt.imshow(item[0], cmap="gray", origin="upper")
plt.title(f"Label: {item[1]}")
plt.axis("off")
plt.tight_layout()

defining a dataloader

In [ ]:
batch_size = 1
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False)

In [ ]:
item[0].shape

## overfitting

defining the model

In [ ]:
model = ae.CNNAutoEncoder()
model = telemetry.ModelTelemetry(
    model,
    loss_names=("total",),
    activation_name_patterns=(".*act.*",),
    gradients_name_patterns=(".*conv\d$",),
    parameter_name_patterns=(".*conv\d$",),
)
model.double()
model.to(device);

In [ ]:
opt = SGD(
    model.parameters(),
    lr=0.1,
)

In [ ]:
loss_func = nn.MSELoss()

In [ ]:
_iter = 0

training loop

In [ ]:
n_epochs = 1_000

model.train()
for epoch in tqdm.tqdm(range(n_epochs), desc="Epochs", total=n_epochs):
    for i, (xb, _) in enumerate(dataloader):
        xb = xb.to(device)
        x_pred = model(xb)

        loss = loss_func(x_pred, xb)

        opt.zero_grad()
        loss.backward()
        opt.step()

        model.loss_history_train(loss, _iter)
        model.parameter_history(_iter)

        _iter += 1

print("Done!")

plotting gradients

In [ ]:
model.draw_gradient_stats(yscale="log", figsize=(12, 20))

plotting activations

In [ ]:
model.draw_activation_stats(yscale="log")

drawing histograms of the weights and biases across training iterations

In [ ]:
model.draw_parameter_stats()

plotting losses

In [ ]:
model.draw_loss_history_train()

inference over samples

In [ ]:
train_features, _ = next(iter(dataloader))

In [ ]:
model.eval();

inspecting predictions

In [ ]:
train_features = train_features.to(device)
preds = model(train_features)
preds[0, :5, :5]

In [ ]:
x_pred = preds.detach().cpu().numpy()
x_pred[0, :3, :5]

In [ ]:
img = train_features[0].cpu()
img_pred = x_pred[0]
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
ax = axs[0]
ax.imshow(img, cmap="gray")
ax.set_title("Input image")
ax.axis("off")
ax = axs[1]
ax.imshow(img_pred, cmap="gray")
ax.set_title("Reconstructed image")
ax.axis("off")
plt.show()

In [ ]:
model.clean_hooks()

In [ ]:
if DO_OVERFITTING_ONLY:
    raise SystemExit("Skipping training beyond overfitting.")

So we can also overfit a single fashion mnist image using this setup, although the picture seems slightly blurry. 

## Reproducing all items

In [ ]:
X0, X1, y0, y1 = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
ds = conv_lecun1990.DigitsDataset(X0, y0)
ds_test = conv_lecun1990.DigitsDataset(X1, y1)

In [ ]:
batch_size = 256
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(ds_test, batch_size=500, shuffle=False)

defining the model

In [ ]:
model = ae.CNNAutoEncoder()
model = telemetry.ModelTelemetry(
    model,
    loss_names=("total",),
    gradients_every_n=100,
    activations_every_n=100,
    parameter_every_n=100,
    activation_name_patterns=(".*act.*",),
    gradients_name_patterns=(".*conv\d",),
    parameter_name_patterns=(".*conv\d",),
)
model.double()
model.to(device);

In [ ]:
opt = SGD(
    model.parameters(),
    lr=0.1,
)

In [ ]:
loss_func = nn.MSELoss()

In [ ]:
_iter = 0

training loop

In [ ]:
n_epochs = 8

model.train()
for epoch in tqdm.tqdm(range(n_epochs), desc="Epochs", total=n_epochs):
    for i, (xb, _) in tqdm.tqdm(
        enumerate(dataloader), desc="Batches", total=len(dataloader)
    ):
        xb = xb.to(device)
        x_pred = model(xb)

        loss = loss_func(x_pred, xb)

        opt.zero_grad()
        loss.backward()
        opt.step()

        model.loss_history_train(loss, _iter)
        model.parameter_history(_iter)

        _iter += 1

    # compute validation loss
    with torch.no_grad():
        model.eval()
        xs_pred, xs_true = [], []
        for xb, _ in dataloader_test:
            xb = xb.to(device)

            x_pred = model(xb)
            xs_pred.append(x_pred)
            xs_true.append(xb)

        x_pred = torch.cat(xs_pred, dim=0)
        x_true = torch.cat(xs_true, dim=0)
        loss_test = loss_func(x_pred, x_true)
        model.loss_history_test(loss_test, _iter)

        model.train()

print("Done!")

plotting gradients

In [ ]:
model.draw_gradient_stats(yscale="log", figsize=(12, 20))

plotting activations

In [ ]:
model.draw_activation_stats(yscale="log")

drawing histograms of the weights and biases across training iterations

In [ ]:
model.draw_parameter_stats()

plotting losses

In [ ]:
model.draw_loss_history_train()

In [ ]:
model.draw_loss_history_test()

In [ ]:
test_features, _ = next(iter(dataloader_test))

In [ ]:
model.eval();

inspecting predictions

In [ ]:
test_features = test_features.to(device)
preds = model(test_features)
preds[0, :5, :5]

In [ ]:
x_pred = preds.detach().cpu().numpy()
x_pred[0, :3, :5]

In [ ]:
def draw_pair(img: torch.Tensor, img_pred: torch.Tensor):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    ax = axs[0]
    ax.imshow(img, cmap="gray")
    ax.set_title("Input image")
    ax.axis("off")
    ax = axs[1]
    ax.imshow(img_pred, cmap="gray")
    ax.set_title("Reconstructed image")
    ax.axis("off")
    plt.show()


def draw_n_pairs(
    input_features: torch.Tensor, x_pred: torch.Tensor, n: int = 5
):
    _n = min(n, len(input_features))
    print(f"Drawing {_n} pairs")
    for i in range(_n):
        img = input_features[i].cpu()
        img_pred = x_pred[i]
        draw_pair(img, img_pred)


draw_n_pairs(test_features, x_pred, n=16)

In [ ]:
model.clean_hooks()